In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup 
import requests
import re

In [2]:
df_matches = pd.read_csv('../data/MatchData_2024.csv',index_col=0)

In [3]:
with pd.option_context('display.max_row',None):
    print(df_matches)

                       Team                Opponent  PointsF  PointsA  G_team  \
0                    Sydney               Melbourne     86.0     64.0    12.0   
1            Brisbane Lions                 Carlton     85.0     86.0    12.0   
2                Gold Coast                Richmond     99.0     60.0    14.0   
3    Greater Western Sydney             Collingwood    114.0     82.0    18.0   
4                   Carlton                Richmond     86.0     81.0    12.0   
5               Collingwood                  Sydney     69.0    102.0    10.0   
6                  Essendon                Hawthorn    107.0     83.0    17.0   
7    Greater Western Sydney         North Melbourne    121.0     82.0    17.0   
8                   Geelong                St Kilda     76.0     68.0    10.0   
9                Gold Coast                Adelaide     60.0     54.0     8.0   
10                Melbourne        Western Bulldogs    109.0     64.0    16.0   
11            Port Adelaide 

Data we need: 
- season_win_rate_team
- season_win_rate_opp
- Last Winner

In [4]:
df_matches_v2 = pd.read_csv('../data/MatchData_v2.csv',index_col=0)

In [5]:
with pd.option_context('display.max_row',None):
    print(df_matches_v2[df_matches_v2['Season']==2023])

                        Team                Opponent  PointsF  PointsA  \
2231                Richmond                 Carlton     58.0     58.0   
2232                 Geelong             Collingwood    103.0    125.0   
2233         North Melbourne              West Coast     87.0     82.0   
2234           Port Adelaide          Brisbane Lions    126.0     72.0   
2235               Melbourne        Western Bulldogs    115.0     65.0   
2236              Gold Coast                  Sydney     61.0    110.0   
2237  Greater Western Sydney                Adelaide    106.0     90.0   
2238                Hawthorn                Essendon     65.0    124.0   
2239                St Kilda               Fremantle     67.0     52.0   
2240                 Carlton                 Geelong     90.0     82.0   
2241          Brisbane Lions               Melbourne     93.0     82.0   
2242             Collingwood           Port Adelaide    135.0     64.0   
2243                Adelaide          

In [6]:
df_matches_comb = pd.concat([df_matches,df_matches_v2[df_matches_v2['Season']<=2023]],axis=0)

In [7]:
df_matches_comb

,Team,Opponent,PointsF,PointsA,G_team,G_opp,Venue,Season,Round,Day,...,H/A,oppH/A,City,Rain (mm),Max Temp (C),Min Temp (C),Max Humid (%),Min Humid (%),Wind Speed (m/s),Solar Rad (MJ/sq m)
0,Sydney,Melbourne,86.0,64.0,12.0,9.0,S.C.G.,2024,Round 1,Thu,...,H,A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Brisbane Lions,Carlton,85.0,86.0,12.0,13.0,Gabba,2024,Round 1,Fri,...,H,A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Gold Coast,Richmond,99.0,60.0,14.0,9.0,Carrara,2024,Round 1,Sat,...,H,A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Greater Western Sydney,Collingwood,114.0,82.0,18.0,11.0,Sydney Showground,2024,Round 1,Sat,...,H,A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Carlton,Richmond,86.0,81.0,12.0,12.0,M.C.G.,2024,Round 2,Thu,...,H,H,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5096,Carlton,Melbourne,73.0,71.0,NaN,NaN,M.C.G.,2023,Semi Final,Fri,...,H,H,Melbourne,0.0,25.6,13.0,73.0,21.0,6.67,18.27
5097,Greater Western Sydney,Port Adelaide,93.0,70.0,NaN,NaN,Adelaide Oval,2023,Semi Final,Sat,...,A,H,Adelaide,0.0,24.0,7.6,88.0,32.0,2.54,19.76
5098,Greater Western Sydney,Collingwood,57.0,58.0,NaN,NaN,M.C.G.,2023,Preliminary Final,Fri,...,A,H,Melbourne,0.2,15.8,3.2,95.0,43.0,3.08,19.12
5099,Carlton,Brisbane Lions,63.0,79.0,NaN,NaN,Gabba,2023,Preliminary Final,Sat,...,A,H,Brisbane,0.0,24.1,14.2,72.0,34.0,2.38,17.68


In [8]:
df_matches_comb['Round'].unique()

array(['Round 1', 'Round 2', 'Round 3', 'Round 4', 'Round 5', 'Round 6',
       'Round 7', 'Round 8', 'Round 9', 'Round 10', 'Round 11',
       'Round 12', 'Round 13', 'Round 14', 'Round 15', 'Round 16',
       'Round 17', 'Round 18', 'Round 19', 'Round 20', 'Round 21',
       'Round 22', 'Round 23', 'Round 24', 'Round 25', 'Qualifying Final',
       'Elimination Final', 'Semi Final', 'Preliminary Final',
       'Grand Final'], dtype=object)

In [9]:
df_matches_comb.loc[(df_matches_comb['Round']=='Qualifying Final')|(df_matches_comb['Round']=='Elimination Final'),'Round'] = 'Finals Week 1'
##Last Winner of the current match up##
    #Rename Round for ease of sorting
round_dict = {'Finals Week 1':25,'Semi Final':26,'Preliminary Final':27,\
            'Grand Final':28}
#Define function to change values based on dict
def rename_round(x):
    if x in list(round_dict.keys()):
        return round_dict[x]
    else:
        return int(re.search(r'(\d+)',x)[0])

#Apply changes to round names
df_matches_comb['RoundNum'] = df_matches_comb['Round'].apply(rename_round)

#Sort entries to appear in chronological order
df_matches_comb = df_matches_comb.sort_values(['Team','Season','RoundNum'])

#Create columns to designate the number in which this match up has appeared
df_matches_comb['MatchUpNum'] = df_matches_comb.groupby(['Team','Opponent']).cumcount()
df_matches_comb['MatchUpP1'] = df_matches_comb['MatchUpNum']+1
#Self join to get the previous winner of the current match up
df_matches_comb = df_matches_comb.merge(df_matches_comb[['Team','Opponent','MatchUpP1','Outcome']],left_on=['Team','Opponent','MatchUpNum'],
               right_on=['Team','Opponent','MatchUpP1'],how='left')
#Remove no longer needed variables
df_matches_comb = df_matches_comb.drop(['MatchUpNum','MatchUpP1_x','MatchUpP1_y'],axis=1)
#Rename columns altered by join
df_matches_comb = df_matches_comb.rename(columns={'Outcome_x':'Outcome','Outcome_y':'Last_Winner'})
df_matches_comb_v2 = df_matches_comb[df_matches_comb['Season']>=2024] 



In [10]:
df_matches_comb_v2['cum_wins'] = df_matches_comb_v2[['Team','Season','Outcome']].groupby(['Team','Season'])['Outcome'].cumsum()
#Get running total number of matches per season per team
df_matches_comb_v2['cum_count'] = df_matches_comb_v2[['Team','Season','Outcome']].groupby(['Team','Season']).cumcount()+1
#Derive running win % and lag by 1
df_matches_comb_v2['season_win_rate'] = (df_matches_comb_v2['cum_wins']/df_matches_comb_v2['cum_count']).shift()
#Remove the variables used in derivation
df_matches_comb_v2 = df_matches_comb_v2.drop(['cum_wins','cum_count'],axis=1)
df_matches_comb_v3 = df_matches_comb_v2[['Team','Opponent','Season','Round','H/A','G_team','G_opp','oppH/A','Outcome','Last_Winner','season_win_rate']]

C:\Users\Adam\AppData\Local\Temp\ipykernel_275676\1081085904.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_matches_comb_v2['cum_wins'] = df_matches_comb_v2[['Team','Season','Outcome']].groupby(['Team','Season'])['Outcome'].cumsum()
C:\Users\Adam\AppData\Local\Temp\ipykernel_275676\1081085904.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_matches_comb_v2['cum_count'] = df_matches_comb_v2[['Team','Season','Outcome']].groupby(['Team','Season']).cumcount()+1
C:\Users\Adam\AppData\Local\Temp\ipy

In [11]:
#Self Join to get opposition season win rate
df_matches_comb_v3 = df_matches_comb_v3.merge(df_matches_comb_v3[['Opponent','Season','Round','season_win_rate']],left_on=['Team','Season','Round']
                         ,right_on=['Opponent','Season','Round'],how='left')

df_matches_comb_v3.drop('Opponent_y',axis=1,inplace=True)

df_matches_comb_v3 = df_matches_comb_v3.rename(columns={'Opponent_x':'Opponent','season_win_rate_x':'season_win_rate_team','season_win_rate_y':'season_win_rate_opp'})

In [12]:
#Get the rolling average number of goals per team
df_matches_comb_v3['G_team_roll'] = df_matches_comb_v3[['Team','G_team']].groupby('Team').rolling(5,min_periods=1).mean().shift().reset_index()['G_team']

In [13]:
#Self Join to get opposition season win rate
df_matches_comb_v3 = df_matches_comb_v3.merge(df_matches_comb_v3[['Opponent','Season','Round','G_team_roll']],left_on=['Team','Season','Round']
                         ,right_on=['Opponent','Season','Round'],how='left')

df_matches_comb_v3.drop(['Opponent_y','G_team_roll_x'],axis=1,inplace=True)

df_matches_comb_v3 = df_matches_comb_v3.rename(columns={'Opponent_x':'Opponent','G_team_roll_y':'G_opp_final'})

In [14]:
df_matches_comb_v3 = df_matches_comb_v3.rename(columns = {'H/A':'HA','oppH/A':'oppHA'})

In [15]:
df_matches_comb_v3

,Team,Opponent,Season,Round,HA,G_team,G_opp,oppHA,Outcome,Last_Winner,season_win_rate_team,season_win_rate_opp,G_opp_final
0,Adelaide,Gold Coast,2024,Round 2,A,8.0,8.0,H,0.0,1.0,NaN,1.00,14.000000
1,Adelaide,Geelong,2024,Round 3,H,11.0,14.0,A,0.0,0.0,0.0,1.00,10.000000
2,Adelaide,Fremantle,2024,Round 4,A,4.0,9.0,H,0.0,1.0,0.0,1.00,14.500000
3,Adelaide,Melbourne,2024,Round 5,H,NaN,NaN,A,NaN,0.0,0.0,0.75,13.500000
4,Adelaide,Carlton,2024,Round 6,A,NaN,NaN,H,NaN,1.0,NaN,NaN,15.333333
...,...,...,...,...,...,...,...,...,...,...,...,...,...
409,Western Bulldogs,Sydney,2024,Round 21,A,NaN,NaN,H,NaN,NaN,NaN,NaN,NaN
410,Western Bulldogs,Melbourne,2024,Round 22,H,NaN,NaN,A,NaN,0.0,NaN,NaN,NaN
411,Western Bulldogs,Adelaide,2024,Round 23,A,NaN,NaN,H,NaN,1.0,NaN,NaN,NaN
412,Western Bulldogs,North Melbourne,2024,Round 24,H,NaN,NaN,H,NaN,NaN,NaN,NaN,NaN


In [16]:
def apply_model(x):
    #intercept
    logit = 0.9503
    #Team HA
    if x.HA == 'H':
        logit += 0.3557
    #Opponent HA    
    if x.oppHA == 'H':
        logit += -0.3049
    
    #G_opp
    if pd.isna(x.G_opp_final): 
        logit += 0 
    else:
        logit += x.G_opp_final*-0.0775
    
    #Season win rate team
    if pd.isna(x.season_win_rate_team):
        logit += 0
    else:
        logit += 2.3101*x.season_win_rate_team
        
    #Season win rate opp
    if pd.isna(x.season_win_rate_opp):
        logit += 0
    else:
        logit += -2.7537*x.season_win_rate_opp
    
    #Last Winner:
    if pd.isna(x.Last_Winner):
        logit += 0
    else:
        logit += x.Last_Winner*0.3777
        
    #Define logit to prob function 
    def logit_to_prob(logit):
        return 1/(1+np.exp(-logit))
    
    return logit_to_prob(logit)
        
        
    

In [17]:
df_matches_comb_v3['Prob'] = df_matches_comb_v3.apply(apply_model,axis=1) 

In [18]:
#Join on the opposite probability
df_matches_comb_v3 = df_matches_comb_v3.merge(df_matches_comb_v3[['Opponent','Season','Round','Prob']],left_on=['Team','Season','Round'],
                                                                                           right_on=['Opponent','Season','Round'],how='left')

df_matches_comb_v3.drop(['Opponent_y'],axis=1,inplace=True)

df_matches_comb_v3 = df_matches_comb_v3.rename(columns={'Opponent_x':'Opponent','Prob_x':'Prob_team','Prob_y':'Prob_opp'})

df_matches_comb_v3['Prob_final_team'] = (df_matches_comb_v3['Prob_team']+(1-df_matches_comb_v3['Prob_opp']))/2 

In [19]:
pd.set_option('display.max_row',None)

In [20]:
df_matches_comb_v3

,Team,Opponent,Season,Round,HA,G_team,G_opp,oppHA,Outcome,Last_Winner,season_win_rate_team,season_win_rate_opp,G_opp_final,Prob_team,Prob_opp,Prob_final_team
0,Adelaide,Gold Coast,2024,Round 2,A,8.0,8.0,H,0.0,1.0,NaN,1.000000,14.000000,0.056487,0.973817,0.041335
1,Adelaide,Geelong,2024,Round 3,H,11.0,14.0,A,0.0,0.0,0.000000,1.000000,10.000000,0.097730,0.937801,0.079965
2,Adelaide,Fremantle,2024,Round 4,A,4.0,9.0,H,0.0,1.0,0.000000,1.000000,14.500000,0.054457,0.946841,0.053808
3,Adelaide,Melbourne,2024,Round 5,H,NaN,NaN,A,NaN,0.0,0.000000,0.750000,13.500000,0.141180,0.896739,0.122220
4,Adelaide,Carlton,2024,Round 6,A,NaN,NaN,H,NaN,1.0,NaN,NaN,15.333333,0.458785,0.670806,0.393990
5,Adelaide,Essendon,2024,Round 7,H,NaN,NaN,A,NaN,0.0,NaN,NaN,14.000000,0.555026,0.605619,0.474704
6,Adelaide,North Melbourne,2024,Round 8,A,NaN,NaN,H,NaN,1.0,NaN,NaN,11.500000,0.532915,0.673652,0.429631
7,Adelaide,Port Adelaide,2024,Round 9,H,NaN,NaN,H,NaN,1.0,NaN,NaN,13.000000,0.591773,0.666212,0.462781
8,Adelaide,Brisbane Lions,2024,Round 10,H,NaN,NaN,A,NaN,0.0,NaN,NaN,NaN,0.786843,0.735576,0.525634
9,Adelaide,Collingwood,2024,Round 11,A,NaN,NaN,H,NaN,0.0,NaN,NaN,NaN,0.655973,0.843394,0.406290


In [21]:
#Extract Rows for upcoming matches
df_matches_comb_v3['Outcome_lag'] = df_matches_comb_v3['Outcome'].shift()

In [22]:
df_final = df_matches_comb_v3[(~df_matches_comb_v3['Outcome_lag'].isna())&(df_matches_comb_v3['Outcome'].isna())&\
                   (df_matches_comb_v3['Round']!='Round 1')][['Team','Opponent','Season','Round','Prob_final_team']]

In [23]:
def team_opp_col(x):
    out_list = [x.Team,x.Opponent]
    out_list.sort()
    return tuple(out_list)

df_final['Team_opp'] = df_final.apply(team_opp_col,axis=1)

In [24]:
df_out = df_final.drop_duplicates('Team_opp').drop('Team_opp',axis=1)

In [25]:
df_out

,Team,Opponent,Season,Round,Prob_final_team
3,Adelaide,Melbourne,2024,Round 5,0.122220
26,Brisbane Lions,North Melbourne,2024,Round 5,0.533669
49,Carlton,Fremantle,2024,Round 5,0.569903
73,Collingwood,Hawthorn,2024,Round 5,0.634604
95,Essendon,Port Adelaide,2024,Round 5,0.360616
141,Geelong,Western Bulldogs,2024,Round 5,0.648936
164,Gold Coast,Greater Western Sydney,2024,Round 5,0.222218
303,Richmond,St Kilda,2024,Round 5,0.407370
349,Sydney,West Coast,2024,Round 5,0.916275
